In [ ]:
# HIDE
import pandas as pd
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True, context="talk")
from IPython import display
%matplotlib inline

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import keras as ke
import keras.backend as K
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import load_model


create_gif = False

# model reconstruction from JSON:
from keras.models import model_from_json

print(f"sklearn: {sk.__version__}")
print(f"pandas: {pd.__version__}")
print(f"kerads: {ke.__version__}")


import src.my_functions_product1 as myFC
import matplotlib.pyplot as plt


In [ ]:
# reading the file from a path and list the field names and input columns of interest

#path = 'data/adult/adult.data'
#output_path = 'Feature-Datasets/adult/'
#arc_id = 'user1'


path = 'data/bank/bank.csv'
output_path = 'Feature-Datasets/bank/'
arc_id = 'user1'

X_df, Ybin, Zbin, result_fname = myFC.read_process_data_output_bias(path)

### Feature Creation

In [ ]:
X_train, X_test, y_train, y_test, Z_train, Z_test = myFC.feature_creation(X_df, Ybin, Zbin)

In [ ]:
feature_train_df = X_train.copy()
feature_train_df['y'] = y_train
feature_train_df['Z'] = Z_train
feature_train_df.to_csv(output_path+'train'+'.csv', index=False)

feature_test_df = X_test.copy()
feature_test_df['y'] = y_test
feature_test_df['Z'] = Z_test
feature_test_df.to_csv(output_path+'test'+'.csv', index=False)




In [ ]:
X_train.head(5)

### Setting up architectures

In [ ]:
#==============================================================================
def User_Classifier_arch(n_features):
    inputs = Input(shape=(n_features,))
    dense1 = Dense(32, activation='relu')(inputs)
    dropout1 = Dropout(0.2)(dense1)
    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)
    dense3 = Dense(32, activation="relu")(dropout2)
    dropout3 = Dropout(0.2)(dense3)
    dense4 = Dense(32, activation="relu")(dropout3)
    dropout4 = Dropout(0.2)(dense4)
    outputs = Dense(1, activation='sigmoid')(dropout4)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model    
#==============================================================================
def User_Adversary_arch(inputs):
    dense1 = Dense(32, activation='relu')(inputs)
    dense2 = Dense(32, activation='relu')(dense1)
    dense3 = Dense(32, activation='relu')(dense2)
    dense4 = Dense(32, activation="relu")(dense3)
    outputs = Dense(1, activation='sigmoid')(dense4)
    return Model(inputs=[inputs], outputs=outputs)   
#==============================================================================
def User_main_task_adv_architecture(n_features):
    main_task_arch = User_Classifier_arch(n_features)
    main_task_arch_json_string = main_task_arch.to_json()
    adv_inputs = Input(shape=(1,))
    adv_task_arch = User_Adversary_arch(adv_inputs)
    adv_task_arch_json_string = adv_task_arch.to_json()
    return main_task_arch_json_string, adv_task_arch_json_string  

## Saving architectures

In [ ]:
n_features = X_train.shape[1]
main_task_arch_json_string, adv_task_arch_json_string = User_main_task_adv_architecture(n_features)

main_task_arch_json_string_file = output_path+arc_id+'_main_task_arch_json_string_file'+'.txt'
with open(main_task_arch_json_string_file,"wt") as out_file:
    out_file.write(main_task_arch_json_string)
    
adv_task_arch_json_string_file = output_path+arc_id+'_adv_task_arch_json_string_file'+'.txt'
with open(adv_task_arch_json_string_file,"wt") as out_file:
    out_file.write(adv_task_arch_json_string)    

## Model and compile only main task: check prediction results

In [ ]:
h5_file_name = output_path+arc_id+'_main_task_ori_trained_model.h5'
save_the_weights = True

main_task_accuracy, p_rule_for_Y1, y_pred = myFC.pre_train_main_task(main_task_arch_json_string, X_train, y_train,X_test,y_test,Z_test, save_the_weights=save_the_weights, h5_file_name= h5_file_name)